<hr style="border-width:2px;border-color:#84C7F7">
<center><h1> Cross-Domain MetaDL Competition </h1></center>
<center><h2>  Any-way Any-shot Learning </h2></center>
<hr style="border-width:2px;border-color:#84C7F7">

Make sure you have installed all the dependencies (`requirements.txt` and `cdmetadl`) in your kernel environment. If you ran the `quick_start.sh` script, make sure you activated the **cdml** conda environment before launching the jupyter notebook. Here is the link of the [Codalab competition](https://codalab.lisn.upsaclay.fr/competitions/3627?secret_key=2d7c4b66-afa5-4c15-92cb-552f8187245c) where you can submit your code and check the leaderboard.


**Outline**: 
- [**I - Data exploration**](#0): Definition of the any-way any-shot learning setup and exploration of how the data is formatted.
- [**II - Submission details**](#1): Explanation of how a submission should be organized.
- [**III - Test and submission**](#2): Example of how to locally test a potential submission and also how to zip your scripts to submit your code on CodaLab. 

<a name='0'></a>
# I - Data exploration

The goal of this section is to familiarize participants with the data format used in the challenge.

Cross-Domain Meta-Learning aims to produce a **Learner** that is able to quickly adapt to new tasks from multiple domains using only a few examples. In the **standard Machine Learning** setting, we usually split the data in train/valid/test sets, these datasets then contain examples assumed to be generated from **the same distribution**. In **within domain few-shot learning**, we have the same idea but with one additional level of abstraction, *i.e.*, instead of regular data splits we have meta-splits (meta-train/meta-valid/meta-test). In this few-shot learning setting, the meta-splits are assumed to have classes generated from **the same task distribution** similarly to standard Machine Learning. However, in the **cross-domain few-shot learning**, the meta-splits are generated from **different task distributions**. To simulate the cross-domain scenario we provide [10 datasets](https://codalab.lisn.upsaclay.fr/competitions/3627?secret_key=2d7c4b66-afa5-4c15-92cb-552f8187245c#participate) during the public phase of this competition. All these datasets belong to different domains and can be used to locally test your potential submissions.

During the competition we generate on the fly the meta-training, meta-validation and meta-testing splits. 
* **Meta-training**: It is used to meta-train your **MetaLearner**, *i.e.*, try to learn the best approach to tackle different tasks from several domains.
* **Meta-validation (optional)**: It can be used to evaluate your **MetaLearner** without worrying about any data leakage.
* **Meta-testing**: We will use it to evaluate your Learner's ability to quickly adapt to new unseen any-way any-shot tasks.

Let's formalize some of the ideas exposed above.

## Definitions

In this competition there are 2 different data formats that can be selected during meta-training. You can either generate data in the form of **tasks** or **batches**. Let's first describe these 2 data formats: 

A **task**, which represents a ***N*-way *k*-shot task**, is described as follows: 
$$ \mathcal{T_j} = \{ \mathcal{D}_{\mathcal{T_j}}^{train}, \mathcal{D}_{\mathcal{T_j}}^{test}\}$$
where $\mathcal{D}_{\mathcal{T_j}}^{train}$ corresponds to the *support set* that contains the training examples for $\mathcal{T_j}$, and $\mathcal{D}_{\mathcal{T_j}}^{test}$ is the *query set* that contains the test examples for $\mathcal{T_j}$. Since this competition is focused on the cross-domain few-shot learning setting, the data contained in one task belongs extrictly to one dataset, but different tasks may come from different datasets because the meta-training split is composed by multiple datasets, *i.e.*, $\mathcal{M}_{\mathcal{D}}^{train} = \{\mathcal{D}_1, \dots, \mathcal{D}_n\}$. The number of datasets ($n$) in the meta-training split $\mathcal{M}_{\mathcal{D}}^{train}$ depends on the number of datasets you want to use for the meta-validation split. During the **public phase** you will have **5 datasets** that you can use for meta-training and meta-validation (*e.g.*  $\mathcal{M}_{\mathcal{D}}^{train} = \{\mathcal{D}_1, \mathcal{D}_2, \mathcal{D}_3\}$ and $\mathcal{M}_{\mathcal{D}}^{valid} = \{\mathcal{D}_4, \mathcal{D}_5\}$) because the remaining 5 datasets are used by the meta-testing split ($\mathcal{M}_{\mathcal{D}}^{test} = \{\mathcal{D}_6, \dots, \mathcal{D}_{10}\}$). However, during the **feedback and final phases** you will have **10 datasets** that you can use for meta-training and meta-validation.


A **batch** is a collection of sampled examples from a dataset **without enforcing a configuration**. Thus, there would be no aforementionned $ \mathcal{D}_{\mathcal{T_j}}^{test}$ unlike the task setting. Note that in the current competition, the data contained in one batch may be sampled from multiple of datasets because we concatenate all meta-training classes and corresponding data of the meta-training split ($\mathcal{M}_{\mathcal{D}}^{train} = \{\mathcal{D}_1, \dots, \mathcal{D}_n\}$) to create a single dataset from which the batches of data are sampled, *i.e.*, $\mathcal{D}^{train} = concat(\mathcal{D}_1, \dots, \mathcal{D}_n)$.

The figure below illustrates the difference between the **task** setting and the **batch** setting.

<center>
<img src="train_settings.png" alt="Train settings" width="700">
</center>


## The any-way any-shot learning problem

The few-shot learning problems are often referred as *N*-way *k*-shots problems. This name refers to the configuration of the tasks at **meta-test time**. The number of **ways** *N* denotes the number of classes in a task that represents an image classification problem. The number of **shots** *k* denotes the number of examples per class in the **support set**. In our case, we focus on the **any-way any-shot** setting. In other words, the tasks at meta-test time represent image classification problems with a number of classes varying from 2 to 20, and the **support set** contains 1 to 20 labeled examples per class, *i.e.*, $N \in [2, 20]$, and $k \in [1, 20]$. Thus, at meta-test time your submission may be tested in the following way:
- **Test task 1:** 5-ways 1-shot task from Dataset 9.
- **Test task 2:** 3-ways 15-shots task from Dataset 3.
- **Test task 3:** 12-ways 4-shots task from Dataset 9.
- **Test task 4:** 2-ways 8-shots task from Dataset 8.
- $\vdots$

Let's summarize the different parts of the meta-learning procedure.

* At **meta-train** time: This is the part you control the most. You can choose to generate data from the meta-train split in the form of **tasks** or **batches**. If you choose the task setting, you can specify the *N*-way *k*-shot configuration for the generated tasks. *N* must to be fixed, but you can define an *N*-way *any*-shot configuration by specifying the boundaries for *k*. Addtionally, you can specify how many images per class you want to have in the *query set* of the generated tasks. On the other hand, if you chose the batch setting, you just have to specify the batch size.
* At **meta-validation** time: You still have control at this stage, but the generated data is always in the form of tasks. Thus, as before, you can specify the *N*-way *k*-shot configuration you want to use including the images per class for the query set. However, for this stage you can define an *any*-way *any*-shot configuration by specifying the boundaries for *N* and *k*.
* At **meta-test** time: You have no control. We always evaluate your submission using any-way any-shot tasks with $N \in [2, 20]$, and $k \in [1, 20]$ and 20 examples per class for the query set. 

As we mentioned previously, in this competition, the tasks/batches are generated **on the fly** from our datasets, but we use the same random seed so each time you make a submission, it will be evaluated with the same data. Also, it is  worth mentioning that the tasks and batches come from **generators**, meaning that there are virtually infinite. 
 
**Note**: Make sure you have downloaded the public datasets under `public_data/` directory in the root directory of this project, i.e. `../cd-metadl`. If you used the `quick_start.sh` script, it automatically downloads the public data. 

Let's see how it looks like in practice.

In [ ]:
# Helpers to visualize the tasks and batches
# DO NOT MODIFY THIS CODE

import os
from collections import Counter

import numpy as np
from torch import Tensor
import matplotlib.pyplot as plt


def plot_task(support_images: Tensor, 
              support_labels: Tensor, 
              query_images: Tensor,
              query_labels: Tensor, 
              size_multiplier: float = 2, 
              max_imgs_per_col: int = 10,
              max_imgs_per_row: int = 10) -> None:
    """ Plots the content of a task. Tasks are composed of a support set 
    (training set) and a query set (test set). 
    
    Args:
        support_images (Tensor): Images in the support set, they have a 
            shape of [support_set_size x channels x height x width].
        support_labels (Tensor): Labels in the support set, they have a 
            shape of [support_set_size]. 
        query_images (Tensor): Images in the query set, they have a 
            shape of [query_set_size x channels x height x width].
        query_labels (Tensor): Labels in the query set, they have a 
            shape of [query_set_size]. 
        size_multiplier (float, optional): Dilate or shrink the size of 
            displayed images. Defaults to 2.
        max_imgs_per_col (int, optional): Number of images in a column. 
            Defaults to 10.
        max_imgs_per_row (int, optional): Number of images in a row. Defaults 
            to 10.
    """
    support_images = np.moveaxis(support_images.numpy(), 1, -1)
    support_labels = support_labels.numpy()
    query_images = np.moveaxis(query_images.numpy(), 1, -1)
    query_labels = query_labels.numpy()

    for name, images, class_ids in zip(("Support", "Query"),
                                     (support_images, query_images),
                                     (support_labels, query_labels)):
        n_samples_per_class = Counter(class_ids)
        n_samples_per_class = {k: min(v, max_imgs_per_col) 
            for k, v in n_samples_per_class.items()}
        id_plot_index_map = {k: i for i, k
            in enumerate(n_samples_per_class.keys())}
        num_classes = min(max_imgs_per_row, len(n_samples_per_class.keys()))
        max_n_sample = max(n_samples_per_class.values())
        figwidth = max_n_sample
        figheight = num_classes
        figsize = (figheight * size_multiplier, figwidth * size_multiplier)
        fig, axarr = plt.subplots(figwidth, figheight, figsize=figsize)
        fig.suptitle(f"{name} Set", size='15')
        fig.tight_layout(pad=3, w_pad=0.1, h_pad=0.1)
        reverse_id_map = {v: k for k, v in id_plot_index_map.items()}
        for i, ax in enumerate(axarr.flat):
            ax.patch.set_alpha(0)
            # Print the class ids, this is needed since, we want to set the x 
            # axis even there is no picture.
            ax.set(xlabel=reverse_id_map[i % figheight], xticks=[], yticks=[])
            ax.label_outer()
        for image, class_id in zip(images, class_ids):
            # First decrement by one to find last spot for the class id.
            n_samples_per_class[class_id] -= 1
            # If class column is filled or not represented: pass.
            if (n_samples_per_class[class_id] < 0 or
                id_plot_index_map[class_id] >= max_imgs_per_row):
                continue
            # If width or height is 1, then axarr is a vector.
            if axarr.ndim == 1:
                ax = axarr[n_samples_per_class[class_id] 
                    if figheight == 1 else id_plot_index_map[class_id]]
            else:
                ax = axarr[n_samples_per_class[class_id], 
                    id_plot_index_map[class_id]]
            ax.imshow(image)
        plt.show()

        
def plot_batch(images: Tensor, 
               labels: Tensor, 
               size_multiplier: int = 1) -> None:
    """ Plot the images in a batch.

    Args:
        images (Tensor): Images inside the batch, they have a shape of 
            [batch_size x channels x height x width].
        labels (Tensor): Labels inside the batch, they have a shape of
            [batch_size].
        size_multiplier (int, optional): Dilate or shrink the size of 
            displayed images. Defaults to 1.
    """
    images = np.moveaxis(images.numpy(), 1, -1)
    labels = labels.numpy()

    num_examples = len(labels)
    figwidth = np.ceil(np.sqrt(num_examples)).astype('int32')
    figheight = num_examples // figwidth
    figsize = (figwidth * size_multiplier, (figheight + 2.5) * size_multiplier)
    _, axarr = plt.subplots(figwidth, figheight, dpi=150, figsize=figsize)

    for i, ax in enumerate(axarr.transpose().ravel()):
        ax.imshow(images[i])
        ax.set(xlabel=str(labels[i]), xticks=[], yticks=[])
    
    plt.show()

First we will show an example of the **task** setting and after that an example of the batch setting.

In [ ]:
from cdmetadl.helpers.general_helpers import prepare_datasets_information
from cdmetadl.ingestion.image_dataset import create_datasets
from cdmetadl.ingestion.data_generator import CompetitionDataLoader

public_data_dir = "../public_data" # Path to Public data
seed = 93 # Random seed to be used

# First we need to read the information from the datasets contained in the 
# pulic data directory 
datasets_info, _, _ = prepare_datasets_information(input_dir=public_data_dir, 
                                                   validation_datasets=0, 
                                                   seed=seed)

# Using the retrieved information we can generate Pytorch Datasets
datasets = create_datasets(datasets_info)

# Now you can define your task configuration 
generator_config = {
    "N": 5, # Number of classes for the generated tasks. If you want to 
            # generate any-way tasks, then put "N": None and define the 
            # boundaries ("min_N" and "max_N")
    "min_N": None,
    "max_N": None,
    "k": 2, # Number of examples per class for the generated tasks. If you want
            # to generate any-shot tasks, then put "k": None and define the 
            # boundaries ("min_k" and "max_k")
    "min_k": None,
    "max_k": None,
    "query_images_per_class": 4
}

# Using the generated Pytorch datasets and the defined configuration we can 
# initialize the data loader
loader = CompetitionDataLoader(datasets=datasets, 
                               episodes_config=generator_config, 
                               seed=seed)

# The initialize data loader has the data generator
generator = loader.generator

In the previous cell, we follow all the required procedure to created a `CompetitionDataLoader` object to extract the data generator. Now you can visualize the configuration you have defined.

In [ ]:
number_of_tasks_to_visualize = 2

for i, task in enumerate(generator(number_of_tasks_to_visualize)):
    print(f"Task {i+1} from Dataset {task.dataset}")
    print(f"# Ways: {task.num_ways}")
    print(f"# Shots: {task.num_shots}")
    plot_task(support_images=task.support_set[0], 
              support_labels=task.support_set[1],
              query_images=task.query_set[0], 
              query_labels=task.query_set[1])

In the figures above, you can observe the composition of a task: A **support set** (train) and a **query set** (test). In the next cell, we present some useful caracteristics of a task.

In [ ]:
print("The task object is organized the following way:\n" 
      + "Task (t):\n"
      + "\t- t.num_ways: int\n"
      + "\t- t.num_shots: int\n"
      + "\t- t.support_set: Tuple[torch.Tensor, torch.Tensor]\n"
      + "\t- t.query_set: Tuple[torch.Tensor, torch.Tensor]\n"
      + "\t- t.original_class_idx: np.ndarray\n"
      + "\t- t.dataset: str")
print(f"\n{'#'*70}\n")
print("The support set images are of the following shape: "
    + f"{task.support_set[0].shape}")
print(f"The support set labels are: {task.support_set[1].unique()} and "
    + f"their shape: {task.support_set[1].shape}")
print(f"\n{'#'*70}\n")
print("The query set images are of the following shape: "
    + f"{task.query_set[0].shape}")
print(f"The query set labels shape is: {task.query_set[1].shape} \n")

Now let's take a look at the **batch** setting. 

Let's assume we would like to receive data from the meta-train split in batches of 20 images. 

In [ ]:
from torch import Generator
from torch.utils.data import DataLoader
from cdmetadl.helpers.general_helpers import prepare_datasets_information
from cdmetadl.helpers.ingestion_helpers import cycle
from cdmetadl.ingestion.image_dataset import ImageDataset
from cdmetadl.ingestion.data_generator import CompetitionDataLoader

public_data_dir = "../public_data" # Path to Public data
seed = 93 # Random seed to be used
batch_size = 20 # You can define the batch size that you want to use

# First we need to read the information from the datasets contained in the 
# pulic data directory 
datasets_info, _, _ = prepare_datasets_information(input_dir=public_data_dir, 
                                                   validation_datasets=0, 
                                                   seed=seed)

# We initialize a random generator to ensure that the same splits are used in
# all submissions
g = Generator()
g.manual_seed(seed)

# As explained before, we concatenate all the datasets of the split
concatenated_dataset = ImageDataset(datasets_info)
num_classes = len(concatenated_dataset.idx_per_label)

# Using the concatenated dataset and the defined batch size we can initialize 
# the data loader
loader = DataLoader(dataset=concatenated_dataset, 
                    batch_size=batch_size, 
                    shuffle=True, 
                    generator=g)

# Lastly, with the initialized data loader we can define our data generator
generator = lambda batches: iter(cycle(batches, loader))

In the previous cell, we follow all the required procedure to created a `DataLoader` object to define our batch data generator. Now you can visualize the batches of data.

In [ ]:
number_of_batches_to_visualize = 2

for i, batch in enumerate(generator(number_of_batches_to_visualize)):
    print(f"Batch {i+1}")
    images, labels = batch
    plot_batch(images, labels)

In the figures above, you can observe the composition of a batch. In the next cell, we present some useful caracteristics of a this data format.

In [ ]:
print("The batch object is organized the following way:\n" 
      + "Batch (b):\n"
      + "\t- b[0]: torch.Tensor (images)\n"
      + "\t- b[1]: torch.Tensor (labels)")
print(f"\n{'#'*70}\n")
print(f"The images are of the following shape: {batch[0].shape}")
print(f"The labels are of the following shape: {batch[1].shape}")
print(f"There is a total of {num_classes} classes in the concatenated dataset."
      +" Thus the batches can contain images from all these classes.")

For the competition you don't need to create your loaders as shown in the previous examples. You will receive the meta-train and meta-valid generators already initialized and ready to use. The way you receive the data generators will be described in the next section. The default setting for the meta-train data is 5-ways any-shot tasks with $k \in [1, 20]$ and 20 images per class for the query set. Similarly, the default setting for the meta-valid data is any-way any-shot tasks with $N \in [2, 20]$, $k \in [1, 20]$ and 20 images per class for the query set. However, if you think you could achieve better performance with your own meta-training and meta-validation setting, you can specify it. In order to specify your own setting, you need to write down your settings in a single json file named `config.json` and put it in your submission folder before zipping it. We will go over the structure of submission folder in the next sections. Here is an example of a config file:

**Content of a `config.json` file**:
```bash
{
    "train_data_format": "task",
    "batch_size": null,
    "train_config": {
        "N": 10,
        "k": null,
        "min_k": 5,
        "max_k": 10,
        "query_images_per_class": 5
    },
    "validation_datasets": 2,
    "valid_config": {
        "N": null,
        "min_N": 2,
        "max_N": 5,
        "k": 20,
        "min_k": null,
        "max_k": null,
        "query_images_per_class": 10
    }
}
```
In the above example configuration you have configured:

* **Meta-training:** 10-ways any-shot tasks with with $k \in [5, 10]$ and 5 images per class for the query set.

* **Meta-validation:** any-way 20-shots tasks with with $N \in [2, 5]$ and 10 images per class for the query set.

Additionally, 2 of the available datasets will be used for the meta-validation split and the remaining datasets will be used for the meta-training split. For clarity the available configurations are:

- `train_data_format`: Format for the training data, it can be "task" or "batch".
- `batch_size`: Batch size for the generated batches. Only used if `train_data_format` is "batch". It cannot be less than 1.
- `train_config`: Configuration for the training data. Only used if `train_data_format` is "task".
  - `N`: Fixed number of ways for the generated tasks at meta-train time. It cannot be less than 2.
  - `k`: Fixed number of shots for the generated tasks at meta-train time. If you would like to use any-shot configuration, then you have to define this parameter as `null`. It cannot be less than 1.
  - `min_k`: Lower bound for the number of shots for the generated tasks at meta-train time. Only used if `k` is `null`. It cannot be less than 1 and must be less than or equal to `max_k`.
  - `max_k`: Upper bound for the number of shots for the generated tasks at meta-train time. Only used if `k` is `null`. It must be greater or equal to `min_k`.
  - `query_images_per_class`: Number of examples per class to include in the query set at meta-train time. It cannot be greater than 20.
- `validation_datasets`: Number of datasets to be used for the meta-valid split. It can be `null`, but in that case, the `meta_valid_generator` that you will receive will be `None`.
- `valid_config`: Configuration for the training data. Only used if `train_data_format` is "task".
  - `N`: Fixed number of ways for the generated tasks at meta-validation time. It cannot be less than 2.
  - `min_N`: Lower bound for the number of ways for the generated tasks at meta-validation time. Only used if `N` is `null`. It cannot be less than 2 and must be less than or equal to `max_N`.
  - `max_N`: Upper bound for the number of ways for the generated tasks at meta-validation time. Only used if `N` is `null`. It must be greater or equal to `min_N`.
  - `k`: Fixed number of shots for the generated tasks at meta-validation time. If you would like to use any-shot configuration, then you have to define this parameter as `null`. It cannot be less than 1.
  - `min_k`: Lower bound for the number of shots for the generated tasks at meta-validation time. Only used if `k` is `null`. It cannot be less than 1 and must be less than or equal to `max_k`.
  - `max_k`: Upper bound for the number of shots for the generated tasks at meta-validation time. Only used if `k` is `null`. It must be greater or equal to `min_k`.
  - `query_images_per_class`: Number of examples per class to include in the query set at meta-validation time. It cannot be greater than 20.

Note that for the meta-training config, you cannot specify `min_N` and `max_N` because as previously explained, during this stage, only fixed number of ways can be used. Additionally, if the configurations that you specify are greater than the maximum number of classes in case of `N` or the maximum number of examples per class in case of `k`, these values will be automatically adjusted to the data available. 

<span style="color:red">**IMPORTANT:**</span> All the datasets have 40 examples per class, but the number of classes varies for each dataset, the minimum is 19 and the maximum is 706.

---

**Section summary** :

* You can choose to generate data from the meta-train split in the form of tasks or batches. Default configurations are tasks but you can change it via a **config.json** file that you put in your folder submission.
* You can choose the configuration for the tasks coming from the meta-validation split (in case you decide to use a meta-validation split). However, we do not allow you to generate batches from this split, only tasks can be generated.

<a name='1'></a>
# II - Submission details
In this section, we will review the structure of a valid submission. We will see that the data we receive for the learning algorithm follows the aforementioned structure.

The participants will have to submit a zip file containing one or several files. The crucial file to add is `model.py`. It contains the meta-learning algorithm logic. This file **must** follow the specific API that we defined for the competition described in the following figure: 

<center>
<img src="API.png" alt="Challenge API" width="500">
</center>

The 3 classes with their associated methods that need to be overwritten are the following:
* **MetaLearner**: The meta-learner contains the meta-algorithm logic. The `meta_fit(meta_train_generator, meta_valid_generator)` method has to be overwritten with your own meta-learning algorithm. It receives the data generators initialized with default setting or your **config.json** file. Note that `meta_valid_generator` is `None` if you define `validation_datasets` as `null`.
* **Learner**: It encapsulates the logic to learn from a new unseen task. Several methods need to be overwritten: 
 * `fit(dataset_train)`: Takes a `dataset_train` as an argument and fit the learner according to it. The `dataset_train` is a tuple that contains the support set images, support set labels, number of ways and number of shots for the current task.
 * `save(path)`: You need to implement a way to save your model in the specified directory. 
 * `load(path)`: You need to implement a way to load your model from the file(s) you created in `save(path)`.
* **Predictor**: The predictor contains the logic of your model to make predictions once the learner is fitted. The `predict(dataset_test)` encapsulates this step. In this case, the `dataset_test` is a `torch.Tensor` which corresponds to the unlabelled query set for the current task.

## Walkthrough a submission example

In this sub-section, we present how your code submission folder should look like before zipping it.  

**Example of a submission directory**
```
random
│   model.py    (Mandatory)
|   metadata    (Mandatory)
|   config.json (Optional but must have this name)
│   helper.py   (Optional) 
│   utils.py    (Optional)
│   ...
```
<code>model.py</code> and <code>metadata</code> are the crucial files to be added. The former is your learning algorithm following our challenge API and the <code>metadata</code> is just a file for the competition server to work properly, you simply add it to your folder without worrying about it (you can find this file in any given baseline's folder). Other files could be added which means that you are free to organize your code as you would like.

## Defining the classes
We will go through a dummy example to understand how to create a model. In the code cell below, you can find the **random** baseline. There are 2 important remarks:
- First, it is mandatory to **write a file(s)** in the `path` given as an argument in the `save(path)` method. It could be a any file, some metadata that you gathered and/or your serialized neural network, but you need to include one.
- Then, one can notice that the shape of the array returned by the `predict` method depends on the query set of each task.
  
**Note**: You can always test your algorithm with `run.py` to verify everything is working properly. We explain how to run the script in the next section.

In [ ]:
""" This is a dummy baseline. It is just supposed to check if ingestion and 
scoring are called properly.
"""

import os
import random
import numpy as np
import pickle
from torch import Tensor
from typing import Iterable, Any, Tuple

from cdmetadl.ingestion.data_generator import Task
from cdmetadl.api.api import MetaLearner, Learner, Predictor

SEED = 98

class MyMetaLearner(MetaLearner):

    def __init__(self, train_classes: int) -> None:
        """ Defines the meta-learning algorithm's parameters. For example, one 
        has to define what would be the learner meta-learner's architecture. 
        
        Args:
            train_classes (int): Total number of classes that can be seen 
                during meta-training. If the data format during training is 
                'task', then this parameter corresponds to the number of ways, 
                while if the data format is 'batch', this parameter corresponds 
                to the total number of classes across all training datasets.
        """
        super().__init__(train_classes)
        self.seed = SEED

    def meta_fit(self, 
                 meta_train_generator: Iterable[Any], 
                 meta_valid_generator: Iterable[Task]) -> Learner:
        """ Uses the generators to tune the meta-learner's parameters. The 
        meta-training generator generates either few-shot learning tasks or 
        batches of images, while the meta-valid generator always generates 
        few-shot learning tasks.
        
        Args:
            meta_train_generator: Function that generates the training data.
                The generated can be a N-way k-shot task or a batch of images 
                with labels.
            meta_valid_generator: Function that generates the validation data.
                The generated data always come in form of N-way k-shot tasks.
                
        Returns:
            Learner: Resulting learner ready to be trained and evaluated on new
                unseen tasks.
        """
        return MyLearner(self.seed)


class MyLearner(Learner):

    def __init__(self, seed: int = 0) -> None:
        """ Defines the learner initialization.

        Args:
            seed (int, optional): Random seed. Defaults to 0.
        """
        super().__init__()
        self.seed = seed

    def fit(self, dataset_train: Tuple[Tensor, Tensor, int, int]) -> Predictor:
        """ Fit the Learner to the support set of a new unseen task. 
        
        Args:
            dataset_train (Tuple[Tensor, Tensor, int, int]): Support set of a 
                task. The data arrive in the following format (X_train, 
                y_train, n_ways, k_shots). X_train is the tensor of labeled 
                imaged of shape [n_ways*k_shots x 3 x 128 x 128], y_train is 
                the tensor of encoded labels (Long) for each image in X_train 
                with shape of [n_ways*k_shots], n_ways is the number of classes 
                and k_shots the number of examples per class.
                        
        Returns:
            Predictor: The resulting predictor ready to predict unlabelled 
                query image examples from new unseen tasks.
        """
        _, y_train, _, _ = dataset_train
        return MyPredictor(y_train, self.seed)

    def save(self, path_to_save: str) -> None:
        """ Saves the learning object associated to the Learner. 
        
        Args:
            path_to_save (str): Path where the learning object will be saved.
        """
        
        if not os.path.isdir(path_to_save):
            raise ValueError(("The model directory provided is invalid. Please"
                + " check that its path is valid."))
        
        pickle.dump(self, open(f"{path_to_save}/learner.pickle", "wb"))
 
    def load(self, path_to_load: str) -> None:
        """ Loads the learning object associated to the Learner. It should 
        match the way you saved this object in self.save().
        
        Args:
            path_to_load (str): Path where the Learner is saved.
        """
        if not os.path.isdir(path_to_load):
            raise ValueError(("The model directory provided is invalid. Please"
                + " check that its path is valid."))
        
        model_file = f"{path_to_load}/learner.pickle"
        if os.path.isfile(model_file):
            with open(model_file, "rb") as f:
                saved_learner = pickle.load(f)
            self.seed = saved_learner.seed
        
    
class MyPredictor(Predictor):

    def __init__(self, 
                 labels: Tensor, 
                 seed: int) -> None:
        """ Defines the Predictor initialization.

        Args:
            labels (Tensor): Tensor of encoded labels.
            seed (int): Random seed.
        """
        super().__init__()
        self.labels = np.unique(labels.numpy())
        random.seed(seed)
        np.random.seed(seed)

    def predict(self, dataset_test: Tensor) -> np.ndarray:
        """ Given a dataset_test, predicts the probabilities associated to the 
        provided images.
        
        Args:
            dataset_test (Tensor): Tensor of unlabelled image examples of shape 
                [n_ways*query_size x 3 x 128 x 128].
        
        Returns:
            np.ndarray: Predicted probs for all images. The array must be of 
                shape [n_ways*query_size, n_ways].
        """
        random_pred = np.random.choice(self.labels, len(dataset_test))
        # Mimic prediction probabilities
        random_probs = np.zeros((random_pred.size, len(self.labels)))
        random_probs[np.arange(random_pred.size), random_pred] = 1
        return random_probs


You can refer to the <code>cd-metadl/baselines/</code> folder if you want to see submission examples. Here are the algorithms provided: 
- The **Random** baseline.
- The **Train from scratch** baseline which learns every task starting from a random initialization at meta-test time, *i.e.*, no meta-learning.  
- The **FineTuning** baseline which pre-trains a network with batches of data from the meta-training split and during meta-testing only fine-tunes the last layer.
- The **Prototypical Networks** based on  [J. Snell et al. - Prototypical Networks for Few-shot Learning (2017)](https://arxiv.org/pdf/1703.05175).
- The **Matching Networks** based on  [O. Vinyals et al. - Matching Networks for One Shot Learning (2017)](https://arxiv.org/abs/1606.04080).
- The **MAML** algorithm based on [C. Finn et al. - Model-Agnostic Meta-Learning for Fast Adaptation of Deep Networks (2017)](https://arxiv.org/pdf/1703.03400).

<a name='2'></a>
# III - Test and Submission

Here we present the `run.py` script. It is meant to mimick what is happenning on the CodaLab platform, i.e. the competition server. Let's say you worked on an algorithm and you are ready to test it before submitting it. More specifically, it will create your MetaLearner object, run the meta-fit method and evaluate your meta-algorithm on test episodes generated from the meta-test split. You can run the script command with the following arguments:
- `input_data_dir`: The path which contains the **public datasets**. 
- `submission_dir`: The path which contains your **algorithm's code** following the format we previously defined. 
- `overwrite_previous_results`: Boolean flag to control if the output folders should be overwritten or not.
- `test_tasks_per_dataset`: Number of tasks per dataset during the meta-testing stage. In the competition, during the feedback phase this parameter is set to 100.

In [ ]:
!python -m cdmetadl.run \
    --input_data_dir=../public_data \
    --submission_dir=../baselines/random \
    --overwrite_previous_results=True \
    --test_tasks_per_dataset=10

## Prepare a ZIP file ready for submission
Here we present how to zip your code to submit it on the CodaLab platform. As an example, we zip the folder <code>cd-metadl/baselines/random/</code> which corresponds to the random baseline which was introduced in the previous section.

In [ ]:
from zip_utils import zipdir

model_dir = "../baselines/random/"
submission_filename = "mysubmission.zip"
zipdir(submission_filename, model_dir)
print(f"Submit this file: {submission_filename}")

## Summary 
For clarity, we summarize the steps that you should be aware of while making a submission : 
- Follow the **MetaLearner**/**Learner**/**Predictor** API to encapsulate your few-shot learning algorithm. Please make sure you name your subclasses as **MyMetaLearner**, **MyLearner** and **MyPredictor** respectively.
- Make sure you <u>save</u> at least a file in the given <code>path</code>. If this is a trained neural network, you need to serialize it in the <code>save()</code> method, and provide code to deserialize it in the <code>load()</code> method. Examples are provided in <code>cd-metadl/baselines/</code>.
- In your algorithm folder, make sure you have <code>model.py</code> and <code>metadata</code> with these **exact** names. If you want to use your custom configuration for the training generator make sure to include the **config.json** file.

--- 

## Next steps
Now you know all the steps required to create a valid code submission.

Good luck !